In [1]:
import pandas as pd

# Read the CSV file
df_ladepunkt = pd.read_csv("../02_data/01_original_data/ladepunktFactTable.csv", sep= ";")

# Display the first few rows of the data

df_ladepunkt.info()
df_ladepunkt.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171805 entries, 0 to 171804
Data columns (total 17 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   ladestation_id                 171805 non-null  int64  
 1   LadeleistungInKW               171805 non-null  float64
 2   Schuko_Stecker                 171805 non-null  bool   
 3   CEE_Stecker_blau               171805 non-null  bool   
 4   CEE_Stecker_rot                171805 non-null  bool   
 5   Typ_1_Stecker                  171805 non-null  bool   
 6   Typ_2_Stecker_Kupplung         171805 non-null  bool   
 7   Typ_2_Stecker_Steckdose        171805 non-null  bool   
 8   CSS_Stecker_Combo_2_Stecker    171805 non-null  bool   
 9   Tesla_Supercharger_V2_Stecker  171805 non-null  bool   
 10  Tesla_Supercharger_V3_Stecker  171805 non-null  bool   
 11  CHAdeMO_Stecker                171805 non-null  bool   
 12  LPArtLSV                      

,ladestation_id,LadeleistungInKW,Schuko_Stecker,CEE_Stecker_blau,CEE_Stecker_rot,Typ_1_Stecker,Typ_2_Stecker_Kupplung,Typ_2_Stecker_Steckdose,CSS_Stecker_Combo_2_Stecker,Tesla_Supercharger_V2_Stecker,Tesla_Supercharger_V3_Stecker,CHAdeMO_Stecker,LPArtLSV,LPArtHPC,LPArtFR,LPArtCharIN,Datenstand
0,1,22.0,False,False,False,False,False,True,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26
1,1,22.0,False,False,False,False,False,True,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26
2,2,22.0,False,False,False,False,False,True,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26
3,2,22.0,False,False,False,False,False,True,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26
4,2,22.0,False,False,False,False,False,True,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26


In [2]:
df_ladepunkt.shape

(171805, 17)

## Feature Engeneering 

In [20]:
import pandas as pd

# Create unique charging point ID
df_ladepunkt["ladepunkt_id"] = df_ladepunkt.groupby("ladestation_id").cumcount() + 1
df_ladepunkt["ladepunkt_id"] = (
    df_ladepunkt["ladestation_id"].astype(str) + "_" + df_ladepunkt["ladepunkt_id"].astype(str)
)

# Aggregate on station level
station_summary = (
    df_ladepunkt.groupby("ladestation_id")
    .agg(
        num_charging_points=("ladepunkt_id", "count"),
        sum_charging_point_capacity_kw=("LadeleistungInKW", "sum"),
        avg_charging_point_capacity_kw=("LadeleistungInKW", "mean"),
        station_capacity_kw=("LadeleistungInKW", "max")  # proxy for station-level capacity
    )
    .reset_index()
)

# Station-based installed capacity (aligned with factsheet definition)
station_summary["station_based_capacity_kw"] = station_summary.apply(
    lambda row: min(row["station_capacity_kw"], row["sum_charging_point_capacity_kw"]),
    axis=1
)

# Preview the result
df_ladepunkt.info()
df_ladepunkt.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171805 entries, 0 to 171804
Data columns (total 18 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   ladestation_id                 171805 non-null  int64  
 1   LadeleistungInKW               171805 non-null  float64
 2   Schuko_Stecker                 171805 non-null  bool   
 3   CEE_Stecker_blau               171805 non-null  bool   
 4   CEE_Stecker_rot                171805 non-null  bool   
 5   Typ_1_Stecker                  171805 non-null  bool   
 6   Typ_2_Stecker_Kupplung         171805 non-null  bool   
 7   Typ_2_Stecker_Steckdose        171805 non-null  bool   
 8   CSS_Stecker_Combo_2_Stecker    171805 non-null  bool   
 9   Tesla_Supercharger_V2_Stecker  171805 non-null  bool   
 10  Tesla_Supercharger_V3_Stecker  171805 non-null  bool   
 11  CHAdeMO_Stecker                171805 non-null  bool   
 12  LPArtLSV                      

,ladestation_id,LadeleistungInKW,Schuko_Stecker,CEE_Stecker_blau,CEE_Stecker_rot,Typ_1_Stecker,Typ_2_Stecker_Kupplung,Typ_2_Stecker_Steckdose,CSS_Stecker_Combo_2_Stecker,Tesla_Supercharger_V2_Stecker,Tesla_Supercharger_V3_Stecker,CHAdeMO_Stecker,LPArtLSV,LPArtHPC,LPArtFR,LPArtCharIN,Datenstand,ladepunkt_id
0,1,22.0,False,False,False,False,False,True,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26,1_1
1,1,22.0,False,False,False,False,False,True,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26,1_2
2,2,22.0,False,False,False,False,False,True,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26,2_1
3,2,22.0,False,False,False,False,False,True,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26,2_2
4,2,22.0,False,False,False,False,False,True,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26,2_3


In [23]:
station_summary.head()

station_summary["sum_charging_point_capacity_kw"].plot


In [24]:
# ===============================
# National-level KPI calculations
# ===============================

# 1. Total number of charging points
total_charging_points = len(df_ladepunkt)

# 2. Total number of stations
total_stations = df_ladepunkt["ladestation_id"].nunique()

# 3. Total installed charging capacity (sum of all charging points)
total_capacity_kw = df_ladepunkt["LadeleistungInKW"].sum()

# 4. Average charging capacity per point
avg_capacity_per_point_kw = df_ladepunkt["LadeleistungInKW"].mean()

# 5. Total station-based installed capacity (from factsheet definition)
total_station_based_capacity_kw = station_summary["station_based_capacity_kw"].sum()

# 6. HPC (High Power Charging) share: charging points >150 kW
hpc_points = (df_ladepunkt["LadeleistungInKW"] > 150).sum()
hpc_share_percent = (hpc_points / total_charging_points) * 100

# 7. Average number of charging points per station
avg_points_per_station = total_charging_points / total_stations

# ===============================
# Print KPI overview
# ===============================
print("⚡ National Charging Infrastructure KPIs ⚡")
print(f"Total charging points: {total_charging_points:,}")
print(f"Total stations: {total_stations:,}")
print(f"Total installed capacity (charging-point based): {total_capacity_kw/1000:,.1f} MW")
print(f"Total installed capacity (station-based): {total_station_based_capacity_kw/1000:,.1f} MW")
print(f"Average capacity per charging point: {avg_capacity_per_point_kw:.1f} kW")
print(f"High Power Charging (HPC >150 kW) points: {hpc_points:,} ({hpc_share_percent:.1f}%)")
print(f"Average charging points per station: {avg_points_per_station:.2f}")


⚡ National Charging Infrastructure KPIs ⚡
Total charging points: 171,805
Total stations: 94,069
Total installed capacity (charging-point based): 10,223.1 MW
Total installed capacity (station-based): 5,905.0 MW
Average capacity per charging point: 59.5 kW
High Power Charging (HPC >150 kW) points: 19,699 (11.5%)
Average charging points per station: 1.83
